```{contents}
```
## Planning vs Execution Agents

### Core Idea

**Planning agents** decide **what should be done and in what order**.
**Execution agents** perform **the actual actions/tools** to complete those steps.

In production-grade agentic systems, these are **separated intentionally**.

---

### Why Split Planning and Execution

Single agents that both plan and act tend to:

* Hallucinate steps
* Skip validations
* Loop endlessly
* Mix reasoning with side effects

Separating them gives:

* Better controllability
* Safer execution
* Easier debugging
* Human-in-the-loop checkpoints

---

### Conceptual Comparison

| Aspect          | Planning Agent | Execution Agent |
| --------------- | -------------- | --------------- |
| Purpose         | Decide steps   | Perform steps   |
| Side effects    | ❌ None         | ✅ Yes           |
| Tools           | ❌ (usually)    | ✅ Required      |
| Determinism     | Low–Medium     | High            |
| Failure impact  | Low            | High            |
| Production role | Orchestrator   | Worker          |

---

### High-Level Architecture

```
User Goal
   ↓
Planning Agent
   ↓  (Plan / Tasks)
Execution Agent
   ↓
Final Result
```

---

### Example Problem

**User request:**

> “Check Jira ticket count, compare with SLA, and notify on-call engineer if breached.”

This requires:

1. Getting ticket count
2. Getting SLA threshold
3. Comparing values
4. Sending notification (conditional)

---

### 1️⃣ Planning Agent

#### Responsibility

* Break the goal into **ordered, explicit steps**
* No API calls
* No side effects

---

#### Planning Agent Prompt (Conceptual)

```text
You are a planning agent.
Given a user goal, produce an ordered list of steps.
Do not execute anything.
Return steps as JSON.
```

---

#### Planning Agent Output

```json
{
  "plan": [
    "Fetch current Jira ticket count",
    "Fetch SLA ticket threshold",
    "Compare ticket count with SLA threshold",
    "If count exceeds SLA, notify on-call engineer"
  ]
}
```

This plan is:

* Explicit
* Auditable
* Modifiable
* Human-reviewable

---

### Execution Agent

#### Responsibility

* Execute steps using **tools**
* Perform side effects
* Be deterministic

---

### Tools (Example)



In [3]:
from langchain_classic.tools import tool

@tool
def ticket_count() -> int:
    '''Returns the current number of open tickets.'''
    return 128

@tool
def sla_threshold() -> int:
    '''Returns the SLA threshold for ticket count.'''
    return 100

@tool
def notify_oncall(message: str) -> str:
    '''Sends a notification to the on-call engineer with the given message.'''
    return "Notification sent"




---

### Execution Agent Logic (Conceptual)



In [7]:
results = {}
results["ticket_count"] = ticket_count.invoke({})
results["sla"] = sla_threshold.invoke({})

if results["ticket_count"] > results["sla"]:
    notify_oncall.invoke({
        "message": f"Ticket count {results['ticket_count']} exceeded SLA {results['sla']}"
    })




---

### Execution Output

```json
{
  "ticket_count": 128,
  "sla": 100,
  "action": "Notification sent"
}
```

---

### Combined Flow (End-to-End)

```
User
 ↓
Planning Agent
 ↓   (JSON Plan)
Execution Agent
 ↓
Result
```

---

### LangChain-Style Demonstration (Conceptual)

#### Planning Chain



In [12]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate

# Initialize LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# Define planning prompt
planning_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a planning agent.
Given a user goal, produce an ordered list of steps.
Do not execute anything.
Return steps as a JSON array with the following structure:
{{
  "plan": [
    "Step 1 description",
    "Step 2 description",
    ...
  ]
}}"""),
    ("human", "{input}")
])


In [13]:
planning_chain = planning_prompt | llm
plan = planning_chain.invoke({
    "input": "Check Jira tickets and notify if SLA breached"
})
print(plan.content)


{
  "plan": [
    "Step 1: Log into the Jira account",
    "Step 2: Navigate to the 'Issues' section",
    "Step 3: Filter the issues based on the SLA status",
    "Step 4: Identify any tickets where the SLA has been breached",
    "Step 5: For each breached ticket, note down the ticket number and details",
    "Step 6: Prepare a notification message including the ticket numbers and details of the breached SLAs",
    "Step 7: Identify the relevant stakeholders to be notified",
    "Step 8: Send the notification message to the identified stakeholders via their preferred method of communication (email, Jira notification, etc.)",
    "Step 9: Confirm that the notification has been sent and received",
    "Step 10: Document the SLA breaches and the notifications sent for future reference"
  ]
}




---

### Execution via AgentExecutor



In [16]:
ans = planning_chain.invoke({
    "input": plan
})
print(ans.content)


{
  "plan": [
    "Step 1: Log into the Jira account",
    "Step 2: Navigate to the 'Issues' section",
    "Step 3: Filter the issues based on the SLA status",
    "Step 4: Identify any tickets where the SLA has been breached",
    "Step 5: For each breached ticket, note down the ticket number and details",
    "Step 6: Prepare a notification message including the ticket numbers and details of the breached SLAs",
    "Step 7: Identify the relevant stakeholders to be notified",
    "Step 8: Send the notification message to the identified stakeholders via their preferred method of communication (email, Jira notification, etc.)",
    "Step 9: Confirm that the notification has been sent and received",
    "Step 10: Document the SLA breaches and the notifications sent for future reference"
  ]
}




Execution agent reads the plan and acts.

---

### Why This Is Safer Than ReAct Alone

| Risk               | ReAct Agent | Plan + Execute |
| ------------------ | ----------- | -------------- |
| Tool misuse        | Medium      | Low            |
| Hallucinated steps | High        | Low            |
| Human review       | Hard        | Easy           |
| Auditing           | Poor        | Excellent      |

---

### Planning Agent vs ReAct Agent

| Aspect    | Planning Agent | ReAct Agent |
| --------- | -------------- | ----------- |
| Planning  | Explicit       | Implicit    |
| Execution | ❌              | ✅           |
| Looping   | ❌              | ✅           |
| Safety    | High           | Medium      |

Planning agents **can be used before ReAct**, but not replaced by it.

---

### Planning + Execution + Human-in-the-Loop

Production pattern:

```
Plan → Human Review → Execute
```

Used in:

* Finance
* Infra automation
* Compliance workflows

---

### Common Mistakes

#### Letting planning agent execute tools

❌ Breaks safety model

#### Letting execution agent invent steps

❌ Hallucination risk

#### No validation between stages

❌ Silent failures

---

### Best Practices

* Planning agent outputs **structured JSON**
* Execution agent only follows plan
* Validate plan before execution
* Separate read and write tools
* Log both plan and execution

---

### When to Use This Pattern

* Multi-step workflows
* Conditional automation
* Risky actions (notifications, writes)
* Enterprise agents

---

### When It’s Overkill

* Simple Q&A
* Single tool calls
* Stateless APIs

---

### Mental Model

```
Planner → decides
Executor → does
LLM → reasons
Tools → act
```

---

### Interview-Ready Summary

> “Planning agents decompose goals into ordered steps without side effects, while execution agents deterministically perform those steps using tools. Separating planning from execution improves safety, auditability, and production reliability.”

---

### Rule of Thumb

* **Think first → Planning agent**
* **Act later → Execution agent**
* **Risky action → Always separate**
* **Production → Plan + Execute**
